In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import re
import itertools

from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.manifold import TSNE
import networkx as nx

import tensorflow as tf
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
import keras 
import scipy
from keras import backend as K
from keras.preprocessing.text import Tokenizer, one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense,Flatten,Activation,Dropout,Input,Embedding,RepeatVector,Permute,merge,Lambda
from keras.layers import multiply ,concatenate ,subtract,add,Add,Reshape                   
from keras.optimizers import RMSprop ,Adam
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.models import Model

Using TensorFlow backend.


In [2]:
train_data = pd.read_csv('/Users/s0c02nj/Desktop/Hike/train/train.csv')
train_feat = pd.read_csv('/Users/s0c02nj/Desktop/Hike/train/user_features.csv')

KeyboardInterrupt: 

In [ ]:
test_data = pd.read_csv('/Users/s0c02nj/Desktop/Hike/test.csv')

Filtering based on class imbalance

In [ ]:
#train_data_c1 = train_data[train_data['is_chat']==1]
#train_data_c1.shape

In [ ]:
#train_data_c0 = train_data[train_data['is_chat']==0]
#train_data_c0.shape
#train_data_c0_mod = train_data_c0.sample(n= 2251784, replace=True, random_state=1)
#train_data_c0_mod.index = range(0,len(train_data_c0_mod))
#train_data_new = pd.concat([train_data_c1,train_data_c0_mod])

All nodes are present

In [ ]:
train_data1 = train_data[['node1_id']]
train_data2 = train_data[['node2_id']]

y_train = train_data['is_chat']

In [ ]:
test_data1 = test_data[['node1_id']]
test_data2 = test_data[['node2_id']]


Bringing the Train Features

In [ ]:
train_feat1 = pd.merge(left = train_data, right= train_feat, how='inner', 
                 left_on=['node1_id'], right_on=['node_id'])

In [ ]:
train_feat2 = pd.merge(left = train_data, right= train_feat, how='inner', 
                 left_on=['node2_id'], right_on=['node_id'])

Bringing Test Features

In [ ]:
test_feat1 = pd.merge(left = test_data1, right= train_feat, how='inner', 
                 left_on=['node1_id'], right_on=['node_id'])

In [ ]:
test_feat2 = pd.merge(left = test_data2, right= train_feat, how='inner', 
                 left_on=['node2_id'], right_on=['node_id'])

Standardizing Columns

In [ ]:
cols_std = ['f1','f2','f3','f4','f5','f5', 'f6','f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13']

In [ ]:
train_feat1 = train_feat1[cols_std]
train_feat2 = train_feat2[cols_std]


In [ ]:
test_feat1 = test_feat1[cols_std]
test_feat2 = test_feat2[cols_std]

In [ ]:
y_train = train_data['is_chat']

In [ ]:
#train_feat1.to_csv('/Users/s0c02nj/Desktop/Hike/train_features_node1.gz', compression='gzip')

In [ ]:
#train_feat2.to_csv('/Users/s0c02nj/Desktop/Hike/train_features_node2.csv')

In [ ]:
#test_feat1.to_csv('/Users/s0c02nj/Desktop/Hike/test_features_node1.csv')

In [ ]:
#test_feat2.to_csv('/Users/s0c02nj/Desktop/Hike/test_features_node2.csv')

Distance Computation

In [ ]:
from scipy.spatial import distance 

edge_dis=[]
for i in range(0,len(train_feat1)):
    #print (i)
    #Defining the features
    a= list(train_feat1.iloc[i])
    b= list(train_feat2.iloc[i])
    
    #Euclidean norm
    dst = distance.euclidean(a, b)
    edge_dis.append(dst)

#### Implementation of Node2Vec

In [ ]:
from node2vec import Node2Vec

In [ ]:
a = list(np.unique(list(train_data['node1_id'].unique()) + list(train_data['node2_id'].unique())))


In [ ]:
x_ones = train_data[train_data['is_chat']==1]
x_ones.index = range(0,len(x_ones))

In [ ]:
x_ls1=[]
for i in range(0,len(x)) :
    x_ls1.append((x['Item1'].iloc[i],x['Item2'].iloc[i],x['Prob'].iloc[i]))

In [ ]:
G = nx.Graph()
G.add_nodes_from(a)

In [ ]:
x_ls1=[]
for i in range(0,len(x_ones)) :
    x_ls1.append((x_ones['node1_id'].iloc[i],x_ones['node2_id'].iloc[i]))

In [ ]:
# add edges from txt 
G.add_edges_from(x_ls1)

In [ ]:
#3# Generate walks
node2vec = Node2Vec(G, dimensions=40, walk_length=16, num_walks=1)

In [ ]:
# Learn embeddings 
model = node2vec.fit(window=10, min_count=1)

In [ ]:
a1 = list(model.wv.get_vector('5931925'))
a2 = list(model.wv.get_vector('4200009'))

In [ ]:
# train_vec1=[]
# train_vec2=[]

# for i in range(0,len(train_data1)):
#     if i%10000==0:
#         print (i)
#     node1 = train_data1['node1_id'].iloc[i]
#     node2 = train_data2['node2_id'].iloc[i]
    
#     train_vec1.append(list(model.wv.get_vector(str(node1))))
#     train_vec2.append(list(model.wv.get_vector(str(node1))))